# Problem Set 3

## Imports

In [1]:
import pandas as pd
from statsmodels.stats.diagnostic import breaks_hansen
import statsmodels.api as sm
#disable warnings
import warnings

warnings.filterwarnings('ignore')

## Exercise 1

In this exercise, you will work with the **Fertility** dataset which
contains data for married women from the 1980 U.S. Census. The dataset
contains information on married women aged 21–35 with two or more
children. Both Excel and Stata versions of the dataset are available on
Moodle (use the one you are more comfortable with)


**Load the data here**


In [2]:
fertility_df = pd.read_stata('fertility.dta')
fertility_df.head()

,morekids,boy1st,boy2nd,samesex,agem1,black,hispan,othrace,weeksm1
0,0,1,0,0,27,0,0,0,0
1,0,0,1,0,30,0,0,0,30
2,0,1,0,0,27,0,0,0,0
3,0,1,0,0,35,1,0,0,0
4,0,0,0,1,30,0,0,0,22



##### a) Regress weeksm1 only on the indicator variable morekids, using OLS. On average, do women with more than two children work less than women with two children? If so, how much?


In [3]:
X = fertility_df['morekids']
y = fertility_df['weeksm1']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                weeksm1   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     3696.
Date:                Fri, 13 Dec 2024   Prob (F-statistic):               0.00
Time:                        15:27:52   Log-Likelihood:            -1.1451e+06
No. Observations:              254654   AIC:                         2.290e+06
Df Residuals:                  254652   BIC:                         2.290e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         21.0684      0.055    385.424      0.0

The coefficient of morekids is -5.387. This means that on average, women with more than two children work 5.387 weeks less than mother with two children. The coefficient is statistically with p-value = 0.

##### b) Explain potential endogeneity issue in the OLS regression estimated in (a).


The morekids variable is potentially endogenous if it is correlated with the error term. This could happen for different reasons such as ommited variable bias, where another variable acts as confounder by influencing both morekids and weeksm1 (i.e. wealth), or simultaneity bias, as weeksm1 could also affect morekids.

##### c) The data set contains the variable samesex, which is equal to 1 if the first two children are of the same sex (boy–boy or girl–girl) and equal to 0 otherwise. Are couples whose first two children are of the same sex more likely to have a third child? Justify your answer based on appropriate regression model.


In [4]:
X = fertility_df['samesex']
y = fertility_df['morekids']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               morekids   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1237.
Date:                Fri, 13 Dec 2024   Prob (F-statistic):          2.23e-270
Time:                        15:27:52   Log-Likelihood:            -1.7673e+05
No. Observations:              254654   AIC:                         3.535e+05
Df Residuals:                  254652   BIC:                         3.535e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3464      0.001    253.791      0.0

The coefficient of samesex is 0.0675. This means that couples whose first two children are of the same sex are 6.75% more likely to have a third child. The coefficient is statistically significant with p-value = 0.

##### d) Is samesex a weak instrument? If so, why? Justify your answer with appropriate statistics.

We can check the strength of the instrument by calculating the F-statistic of the first stage regression. If the F-statistic is less than 10, then the instrument is weak. We can also check the first stage regression summary to see if the coefficient of the instrument is statistically significant.

In [5]:
print(f"F-statistic: {model.fvalue}")

F-statistic: 1237.219435998433


The F-statistic is 1237. This is much higher than 10, so the instrument is not weak. The coefficient of the instrument is also statistically significant with p-value = 0.




##### e) How can the exclusion restriction condition be violated when using samesex as an instrument? Discuss it.


The exclusion restriction condition requires that the instrument (samesex) affects the outcome variable (weeksm1) only through its effect on the endogenous variable (morekids) and not through any other channel. This condition could be violated if having two children of the same sex directly influences the mother worked weeks or if it is correlated with unobserved factors that also affect weeksm1, which would act as confounders. Such violations would lead to biased IV estimates. At first glance, it doesn't look likely.

##### f) Estimate the IV regression of weeksm1 on morekids, using samesex as an instrument. How large is the fertility effect on labor supply?


In [6]:
from linearmodels.iv import IV2SLS

formula = 'weeksm1 ~ 1 + [morekids ~ samesex]'
iv = IV2SLS.from_formula(formula, fertility_df).fit()
print(iv)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                weeksm1   R-squared:                      0.0139
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0139
No. Observations:              254654   F-statistic:                    24.534
Date:                Fri, Dec 13 2024   P-value (F-stat)                0.0000
Time:                        15:27:53   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      21.421     0.4872     43.963     0.0000      20.466      22.376
morekids      -6.3137     1.2747    -4.9532     0.00

The coefficient of morekids is -6.314. This means that on average, women with more than two children work 6.314 weeks less than mother with two children. The coefficient is statistically with p-value = 0.

##### g) Do the magnitude and significancy of the IV estimation results (the coefficient for morekids) change when you include the variables agem1, black, hispan, and othrace in the regression in part f (treating these variables as exogenous)? Explain why or why not.

In [7]:
formula = 'weeksm1 ~ 1 + [morekids ~ samesex] + agem1 + black + hispan + othrace'
iv = IV2SLS.from_formula(formula, fertility_df).fit()
print(iv)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                weeksm1   R-squared:                      0.0437
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0437
No. Observations:              254654   F-statistic:                    6955.0
Date:                Fri, Dec 13 2024   P-value (F-stat)                0.0000
Time:                        15:27:54   Distribution:                  chi2(5)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept     -4.7919     0.3898    -12.294     0.0000     -5.5559     -4.0279
agem1          0.8316     0.0226     36.730     0.00

The coefficient of morekids is -5.82 and is still statistically significant with p-value = 0.  The magnitude of the coefficient lowered slightly compared to the previous model. This can be explained by the fact that the exogenous variables included in the model are maybe correlated with the endogenous variable morekids through the instrumental variable. Despite the IV regression should account for the endogeneity, and adress the omitted variable bias, adding new information (exogenous variables) refines this adjustment. The coefficient change might also not be significant.

## Exercise 2

In this exercise, you will work with a simplified version of
**SchoolingReturns** dataset from the U.S. National Longitudinal Survey
of Young Men (NLSYM) conducted in 1976, with some variables dating back
to earlier years. The dataset captures information on individuals’
wages, education levels, labor market experience, ethnicity, and other
demographic factors (we limited dataset to only necessary variables for
the exercise). This dataset has been utilized in studies examining the
causal relationship between education and earnings, notably by David
Card (1995), who employed geographical proximity to colleges as an
instrumental variable to estimate the returns to schooling. Both Excel
and Stata versions of the dataset are available on Moodle (use the one
you are more comfortable with).

The variable description is as follows:

-   *educ: Education level*

-   *momeduc: Mothers’ education level*

-   *dadeduc: Dads’ education level*

-   *lwage: Log wage*

-   *smsa: Indicator for whether an individual resided in an urban area
    (as defined by the U.S. Census Bureau) at the time of the survey.
    The value is one for individuals living in an urban/metropolitan
    area and zero for individuals living in a non-urban/rural area*

-   *black dummy: Indicator for black individuals*

-   *south dummy: Indicates whether an individual resided in the
    Southern United States at the time of the survey. We use it as a
    control to capture regional differences in labor markets, education
    access, and wage structures*

-   *nearc4 dummy: Indicator for whether an individual lived near a
    four-year college during their youth*

-   *age: individual age*

-   *age squared: individual age squared*

In all models, specify individuals and their parent education as a
continuous variable not a categorical variable.

**Load the data here**

In [8]:
school_returns_df = pd.read_stata('ReturnSchooling.dta')
school_returns_df.head()

,educ,age,dadeduc,momeduc,lwage,smsa,black_dummy,south_dummy,nearc4_dummy,age_squared
0,12,27,8.0,8.0,6.175867,1,0,0,0,729.0
1,12,34,14.0,12.0,6.580639,1,0,0,0,1156.0
2,11,27,11.0,12.0,5.521461,1,0,0,1,729.0
3,12,34,8.0,7.0,6.591674,1,0,0,1,1156.0
4,12,26,9.0,12.0,6.214608,1,0,0,1,676.0



##### a) Run an OLS regression using lwage as the dependent variable and educ, black dummy, south dummy, age, age squared, and smsa as independent variables. Interpret the education level significancy and the estimated coefficient.


In [9]:
X = school_returns_df[['educ', 'black_dummy', 'south_dummy', 'age', 'age_squared', 'smsa']]
y = school_returns_df['lwage']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.211
Model:                            OLS   Adj. R-squared:                  0.208
Method:                 Least Squares   F-statistic:                     90.04
Date:                Fri, 13 Dec 2024   Prob (F-statistic):          2.36e-100
Time:                        15:27:54   Log-Likelihood:                -869.61
No. Observations:                2033   AIC:                             1753.
Df Residuals:                    2026   BIC:                             1793.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           3.5965      0.829      4.337      

The coefficient of educ is 0.0305. This means that a one-year increase in education level leads to a 3.05% increase in wage. The coefficient is statistically significant with p-value = 0.




We know that the estimated effect of schooling on future wage can be endogenous. One example is ‘ability bias’ (see Griliches, 1977). Suppose that some individuals have unobserved characteristics (ability) that enable them to get higher earnings. If these individuals also have above-average schooling levels, this implies a positive correlation between schooling and error term and an OLS estimator that is upward biased. Another reason for endogeneity is the existence of measurement error in the schooling measure which leads to downward bias in the OLS estimator. Therefore, we decide to implement IV estimation to alleviate the potential bias.

##### b) Implement TSLS using nearc4 dummy as instrument for educ (Do not perform this manually. Use the packages!). Put the same exogenous variables as part a. Compare the estimated coefficient and standard error for educ of this model with part a. Specifically, is the OLS regression upwardly or downwardly biased? which model suggests higher standard error and why?


In [10]:
formula = 'lwage ~ 1 + [educ ~ nearc4_dummy] + black_dummy + south_dummy + age + age_squared + smsa'
iv = IV2SLS.from_formula(formula, school_returns_df).fit()
print(iv)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                      0.1760
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1735
No. Observations:                2033   F-statistic:                    495.79
Date:                Fri, Dec 13 2024   P-value (F-stat)                0.0000
Time:                        15:27:54   Distribution:                  chi2(6)
Cov. Estimator:                robust                                         
                                                                              
                              Parameter Estimates                              
             Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------
Intercept       3.5186     0.8431     4.1734     0.0000      1.8661      5.1711
black_dummy    -0.1331     0.0660    -2.0156    

The coefficient of educ is 0.0653 with std. Error 0.0554 and p-value = 0.2384. The standard error of educ in the IV model is way higher than in the OLS model, and the coefficient is not statistically significant with p-value > 0.05. If we consider the given coefficient despite its' probability to be null, the OLS regression is downwardly biased, as the IV model suggests a higher coefficient. Otherwise, we can consider the coefficient to be 0 and the OLS regression to be upwardly biased. The IV model has higher standard error because the IV method accounts for endogeneity by using instruments to predict the endogenous variable, which introduces additional variability. This additional uncertainty about the instrument's validity and its correlation with the endogenous variable makes the IV estimates less precise than OLS estimates. The large value also suggest that the instrument is weak.

##### c) Now implement TSLS again, using momeduc and dadeduc as instruments for educ and report the results.

In [11]:
formula = 'lwage ~ 1 + [educ ~ momeduc + dadeduc] + black_dummy + south_dummy + age + age_squared + smsa'
iv = IV2SLS.from_formula(formula, school_returns_df).fit()
print(iv)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                      0.2086
Estimator:                    IV-2SLS   Adj. R-squared:                 0.2062
No. Observations:                2033   F-statistic:                    523.92
Date:                Fri, Dec 13 2024   P-value (F-stat)                0.0000
Time:                        15:27:54   Distribution:                  chi2(6)
Cov. Estimator:                robust                                         
                                                                              
                              Parameter Estimates                              
             Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------
Intercept       3.5781     0.8189     4.3693     0.0000      1.9730      5.1831
black_dummy    -0.1630     0.0271    -6.0096    

The coefficient of educ is 0.0387 with std. Error 0.0106 and p-value = 0.0003.

Now we want to compare model in part b and c.


##### d) Which model has weaker instrument(s), model b or c? Justify your answer using appropriate statistics?


We can check the strength of the instrument by calculating the F-statistic of the first stage regression. If the F-statistic is less than 10, then the instrument is weak. We can also check the first stage regression summary to see if the coefficient of the instrument is statistically significant.

In [12]:
formula = 'educ ~ 1 + nearc4_dummy'
iv_b = IV2SLS.from_formula(formula, school_returns_df).fit()
print(iv_b)

                            OLS Estimation Summary                            
Dep. Variable:                   educ   R-squared:                      0.0087
Estimator:                        OLS   Adj. R-squared:                 0.0083
No. Observations:                2033   F-statistic:                    18.759
Date:                Fri, Dec 13 2024   P-value (F-stat)                0.0000
Time:                        15:27:54   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                              Parameter Estimates                               
              Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------------------------
Intercept        13.598     0.0894     152.17     0.0000      13.423      13.773
nearc4_dummy     0.4677     0.1080     4.331

In [13]:
formula = 'educ ~ 1 + momeduc + dadeduc'
iv_c = IV2SLS.from_formula(formula, school_returns_df).fit()
print(iv_c)

                            OLS Estimation Summary                            
Dep. Variable:                   educ   R-squared:                      0.1658
Estimator:                        OLS   Adj. R-squared:                 0.1650
No. Observations:                2033   F-statistic:                    441.69
Date:                Fri, Dec 13 2024   P-value (F-stat)                0.0000
Time:                        15:27:54   Distribution:                  chi2(2)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      10.083     0.1987     50.755     0.0000      9.6934      10.472
momeduc        0.1666     0.0208     8.0000     0.00

In [14]:
print(f"F-statistic of model b: {iv_b.f_statistic.stat}")
print(f"F-statistic of model c: {iv_c.f_statistic.stat}")

F-statistic of model b: 18.759033936857932
F-statistic of model c: 441.6884302890728


The F-stat of model c is way higher than model b, so model b has weaker instruments. We could expect this results from the standard error of educ in model b being higher than in model c.

##### e) Compare standard error of educ in model b and c. Why are they very different?

The standard error of educ in model c is 0.0106, while in model b is 0.0554. The standard error of educ in model b is higher than in model c because the instrument in model c is stronger than in model b. The stronger the instrument, the more precise the estimates are, and the lower the standard error.

##### f) Test the overidentifying restriction using Hansen J-test for model c (use the steps discussed in the exercise sessions). Is it significant at the 1% level? What does significancy or insignificancy of the J-statistic mean?


In [31]:
from scipy.stats import chi2

num_instruments = 2
num_endog = 1
degrees_of_freedom = num_instruments - num_endog
n = len(school_returns_df)

# Overidentifying restrictions test (J-statistic)
# Step 1: Residual regression
school_returns_df['residuals'] = iv.resids
residuals_formula = 'residuals ~ 1 + momeduc + dadeduc + black_dummy + south_dummy + age + age_squared + smsa'
residuals_model = sm.OLS.from_formula(residuals_formula, data=school_returns_df).fit()

# Step 2: Compute the J-statistic

# Method 1: Using the Wald test

# We then test whether the coefficients on the instruments are = 0
hypotheses = ['momeduc = 0', 'dadeduc = 0']
test_result = residuals_model.wald_test(hypotheses)
j_statistic = test_result.statistic[0][0]

p_value = chi2.sf(j_statistic, df=degrees_of_freedom)

print(f"Wald test J-statistic: {j_statistic}")
print(f"Wald test p-value: {p_value}")

# Method 2: Using the formula

j_statistic = n * residuals_model.rsquared

p_value = chi2.sf(j_statistic, df=degrees_of_freedom)

print(f"Formula J-statistic: {j_statistic}")
print(f"Formula p-value: {p_value}")

Wald test J-statistic: 2.8801246013599444
Wald test p-value: 0.0896790821721795
Formula J-statistic: 5.7666022533825005
Formula p-value: 0.016333585041913037


I will consider the p_value from Wald test such as in the exercises. The p_value is 0.09 which is greater than 0.01, so it is not significant and we fail to reject the null hypothesis. Significancy means that we reject the null hypothesis that the instruments are valid, thus at least one instrument is invalid and correlated with error term. Insignificancy means that we fail to reject the null hypothesis, so the instruments are valid and uncorrelated with the error term. It is the case here and our instruments and model are valid.




##### g) Which model do you prefer? b or c? Explain based on what you have found in previous steps.


I would prefer model c as we have seen that it has stronger instruments, lower standard error and its' overidentifying restriction test is not significant, which meant that the instruments are valid and uncorrelated with the error term, so the IV estimates are consistent and unbiased. The model c is valid and better than b.